In [ ]:
import resources.workspace as ws
%matplotlib inline
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
plt.ion();

$
% START OF MACRO DEF
% DO NOT EDIT IN INDIVIDUAL NOTEBOOKS, BUT IN macros.py
%
\newcommand{\Reals}{\mathbb{R}}
\newcommand{\Expect}[0]{\mathbb{E}}
\newcommand{\NormDist}{\mathcal{N}}
%
\newcommand{\DynMod}[0]{\mathscr{M}}
\newcommand{\ObsMod}[0]{\mathscr{H}}
%
\newcommand{\mat}[1]{{\mathbf{{#1}}}} 
%\newcommand{\mat}[1]{{\pmb{\mathsf{#1}}}}
\newcommand{\bvec}[1]{{\mathbf{#1}}} 
%
\newcommand{\trsign}{{\mathsf{T}}} 
\newcommand{\tr}{^{\trsign}} 
\newcommand{\tn}[1]{#1} 
\newcommand{\ceq}[0]{\mathrel{≔}}
%
\newcommand{\I}[0]{\mat{I}} 
\newcommand{\K}[0]{\mat{K}}
\newcommand{\bP}[0]{\mat{P}}
\newcommand{\bH}[0]{\mat{H}}
\newcommand{\bF}[0]{\mat{F}}
\newcommand{\R}[0]{\mat{R}}
\newcommand{\Q}[0]{\mat{Q}}
\newcommand{\B}[0]{\mat{B}}
\newcommand{\C}[0]{\mat{C}}
\newcommand{\Ri}[0]{\R^{-1}}
\newcommand{\Bi}[0]{\B^{-1}}
\newcommand{\X}[0]{\mat{X}}
\newcommand{\A}[0]{\mat{A}}
\newcommand{\Y}[0]{\mat{Y}}
\newcommand{\E}[0]{\mat{E}}
\newcommand{\U}[0]{\mat{U}}
\newcommand{\V}[0]{\mat{V}}
%
\newcommand{\x}[0]{\bvec{x}}
\newcommand{\y}[0]{\bvec{y}}
\newcommand{\z}[0]{\bvec{z}}
\newcommand{\q}[0]{\bvec{q}}
\newcommand{\br}[0]{\bvec{r}}
\newcommand{\bb}[0]{\bvec{b}}
%
\newcommand{\bx}[0]{\bvec{\bar{x}}}
\newcommand{\by}[0]{\bvec{\bar{y}}}
\newcommand{\barB}[0]{\mat{\bar{B}}}
\newcommand{\barP}[0]{\mat{\bar{P}}}
\newcommand{\barC}[0]{\mat{\bar{C}}}
\newcommand{\barK}[0]{\mat{\bar{K}}}
%
\newcommand{\D}[0]{\mat{D}}
\newcommand{\Dobs}[0]{\mat{D}_{\text{obs}}}
\newcommand{\Dmod}[0]{\mat{D}_{\text{obs}}}
%
\newcommand{\ones}[0]{\bvec{1}} 
\newcommand{\AN}[0]{\big( \I_N - \ones \ones\tr / N \big)}
%
% END OF MACRO DEF
$
## Dynamical systems
are systems (sets of equations) whose variables evolve in time (the equations contains time derivatives). As a branch of mathematics, its theory is mainly concerned with understanding the behaviour of solutions (trajectories) of the systems.

## Chaos
is also known as the butterfly effect: "a butterfly that flaps its wings in Brazil can 'cause' a hurricane in Texas".

As opposed to the opinions of Descartes/Newton/Laplace, chaos effectively means that even in a deterministic (non-stochastic) universe, we can only predict "so far" into the future. This will be illustrated below using two toy-model dynamical systems made by Edward Lorenz.

---
## The Lorenz (1963) attractor

The [Lorenz-63 dynamical system](https://en.wikipedia.org/wiki/Lorenz_system) can be derived as an extreme simplification of *Rayleigh-Bénard convection*: fluid circulation in a shallow layer of fluid uniformly heated (cooled) from below (above).
This produces the following 3 *coupled* ordinary differential equations (ODE):

$$
\begin{aligned}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{aligned}
$$

where the "dot" represents the time derivative, $\frac{d}{dt}$. The state vector is $\x = (x,y,z)$, and the parameters are typically set to

In [ ]:
SIGMA = 10.0
BETA  = 8/3
RHO   = 28.0

The ODEs can be coded as follows

In [ ]:
def dxdt(xyz, t0, sigma, beta, rho):
    """Compute the time-derivative of the Lorenz-63 system."""
    x, y, z = xyz
    return [
        sigma * (y - x),
        x * (rho - z) - y,
        x * y - beta * z
    ]

#### Numerical integration to compute the trajectories

Below is a function to numerically **integrate** the ODEs and **plot** the solutions.

<!--
This function also takes arguments to control ($\sigma$, $\beta$, $\rho$) and of the numerical integration (`N`, `T`).
-->

In [ ]:
from scipy.integrate import odeint
output_63 = [None]

@ws.interact(      sigma=(0.,50), beta=(0.,5), rho=(0.,50), N=(0,50), eps=(0.01,1), T=(0.,40))
def animate_lorenz(sigma=SIGMA,   beta=BETA,   rho=RHO    , N=2,      eps=0.01,     T=1.0):    
    
    # Initial conditions: perturbations around some "proto" state
    rnd.seed(1)
    x0_proto = np.array([-6.1, 1.2, 32.5])
    x0 = x0_proto + eps*rnd.randn(N, 3)

    # Compute trajectories
    tt = np.linspace(0, T, int(100*T)+1)              # Time sequence for trajectory
    dd = lambda x, t: dxdt(x, t, sigma, beta, rho)    # Define dxdt(x,t) with fixed params.
    xx = np.array([odeint(dd, xn, tt) for xn in x0])  # Integrate
    output_63[0] = xx
    
    # PLOTTING
    ax = plt.figure(figsize=(10, 5)).add_subplot(111, projection='3d')
    ax.axis('off')
    colors = plt.cm.jet(np.linspace(0, 1, N))
    for n in range(N):
        ax.plot(*(xx[n, :, :].T), '-',    color=colors[n])
        ax.scatter3D(*xx[n, -1, :], s=40, color=colors[n])
        plt.show()

**Exc 2**: 
* Move `T` (use your arrow keys). What does it control?  
* Set `T` to something small; move the sliders for `N` and `eps`. What do they control?
* Visually investigate the system's (i.e. the trajectories') sensitivity to initial conditions by moving `T`, `N` and `eps`. How long do you think it takes (on average) for two trajectories (or the estimation error) to grow twice as far apart as they started (alternatives: 0.03, 0.3, 3, 30)?

### Averages

Slide `N` and `T` to their upper bounds. Execute the code cell below.

In [ ]:
# Compute the average location of the $m$-th component of the state in TWO ways.

m  = 0 # state component index (must be 0,1,2)
nB = 20
xx = output_63[0][:, :, m] 

plt.hist(xx[:, -1]  , density=1, bins=nB, label="ensemble dist."        ,alpha=1.0) # -1: last time
plt.hist(xx[-1, :]  , density=1, bins=nB, label="temporal dist."        ,alpha=0.5) # -1: last ensemble member
#plt.hist(xx.ravel(), density=1, bins=nB, label="total    distribution", alpha=0.5)
plt.legend();

**Exc 6*:** Answer the questions below.
 * (a) Do you think the samples behind the histograms are drawn from the same distribution?
 * (b) The answer to the above question means that this dynamical system is [ergodic](https://en.wikipedia.org/wiki/Ergodic_theory#Ergodic_theorems).
 Now, suppose we want to investigate which (DA) method is better at estimating the true state (trajectory) for this system, on average. Should we run several short experiments or one long one?

In [ ]:
#ws.show_answer("Ergodicity a")
#ws.show_answer("Ergodicity b")

---

## The "Lorenz-95" model

The Lorenz-96 system
is a "1D" model, designed to resemble atmospheric convection.  Each state variable $\x_m$ can be considered some atmospheric quantity at grid point at a fixed latitude of the earth.  The system 
is given by the coupled set of ODEs,
$$
\frac{d \x_m}{dt} = (\x_{m+1} − \x_{m-2}) \x_{m-1} − \x_m + F
\, ,
\quad \quad m \in \{1,\ldots,M\}
\, ,
$$
where the subscript indices apply periodically.

This model is not derived from physics but has similar characteristics, such as
<ul>
    <li> there is external forcing, determined by a parameter $F$;</li>
    <li> there is internal dissipation, emulated by the linear term;</li>
    <li> there is energy-conserving advection, emulated by quadratic terms.</li>
</ul>

[Further description in the very readable original article](http://eaps4.mit.edu/research/Lorenz/Predicability_a_Problem_2006.pdf).

**Exc 10:** Show that the "total energy" $\sum_{m=1}^{M} \x_m^2$ is preserved by the quadratic terms in the ODE.  


In [ ]:
#ws.show_answer("Hint: Lorenz energy")
#ws.show_answer("Lorenz energy")

The model is animated below.

In [ ]:
# For all m, any n: s(x, n) := x[m+n], circularly.
def s(x, n):
    return np.roll(x, -n)

output_95 = [None]

@ws.interact(         M=(5,60,1), Force=(0,40,1), eps=(0.01,3,0.1), T=(0.05,40,0.05))
def animate_lorenz_95(M=40,       Force=8.0,      eps=0.01,         T=0):
    # Initial conditions: perturbations
    x0 = np.zeros(M)
    x0[0] = eps
    
    def dxdt(x, t):
        return (s(x, 1) - s(x, -2)) * s(x, -1) - x + Force
    
    tt = np.linspace(0, T, int(40*T)+1)
    xx = odeint(lambda x, t: dxdt(x, t), x0, tt)
    output_95[0] = xx
    
    plt.figure(figsize=(7, 4))
    
    # Plot last only
    #plt.plot(xx[-1], 'b')

    # Plot multiple
    Lag = 8
    colors = plt.cm.cubehelix(0.1 + 0.6*np.linspace(0, 1, Lag))
    for k in range(Lag, 0, -1):
        plt.plot(xx[max(0, len(xx)-k)], color=colors[Lag-k])
    plt.ylim(-10, 20)
    plt.show()

**Exc 12:** Investigate by moving the sliders: Under which settings of the force `F` is the system chaotic (is the predictability horizon finite)?

---
## Error/perturbation dynamics

**Exc 14*:** Suppose $x(t)$ and $z(t)$ are "twins": they evolve according to the same law $f$:
$$
\begin{align}
\frac{dx}{dt} &= f(x) \\
\frac{dz}{dt} &= f(z) \, .
\end{align}
$$

Define the "error": $\varepsilon(t) = x(t) - z(t)$.  
Suppose $z(0)$ is close to $x(0)$.  
Let $F = \frac{df}{dx}(x(t))$.  

* a) Show that the error evolves according to the ordinary differential equation (ODE)
$$\frac{d \varepsilon}{dt} \approx F \varepsilon \, .$$ 

In [ ]:
#ws.show_answer("error evolution")

* b) Suppose $F$ is constant. Show that the error grows exponentially: $\varepsilon(t) = \varepsilon(0) e^{F t} $.

In [ ]:
#ws.show_answer("anti-deriv")

* c)
 * 1) Suppose $F<0$.  
 What happens to the error?  
 What does this mean for predictability?
 * 2) Now suppose $F>0$.  
 Given that all observations are uncertain (i.e. $R_t>0$, if only ever so slightly),  
 can we ever hope to estimate $x(t)$ with 0 uncertainty?
 

In [ ]:
#ws.show_answer("predictability cases")

* d) Consider the ODE derived above.  
How might we change it in order to model (i.e. emulate) a saturation of the error at some level?  
Can you solve this equation?

In [ ]:
#ws.show_answer("saturation term")

* e) Now suppose $z(t)$ evolves according to $\frac{dz}{dt} = g(z)$, with $g \neq f$.  
What is now the differential equation governing the evolution of the error, $\varepsilon$?

In [ ]:
#ws.show_answer("liner growth")

**Exc 16*:** Recall the Lorenz-63 system. What is its doubling time (i.e. estimate how long does it take for two trajectories to grow twice as far apart as they were to start with) ?
*Hint: Set `N=50, eps=0.01, T=1,` and compute the spread of the particles now as compared to how they started*

In [ ]:
xx = output_63[0][:, -1] # Ensemble of particles at the end of integration
### compute your answer here ###

In [ ]:
#ws.show_answer("doubling time")

The answer actually depends on where in "phase space" the particles started.
To get a universal answer one must average these experiments for many different initial conditions.

## In summary: 
Prediction (forecasting) with these systems is challenging because they are chaotic: small errors grow exponentially.

In other words, chaos means that there is a limit to how far into the future we can make predictions (skillfully).

It is therefore crucial to minimize the initial error as much as possible. This is a task for DA.

### Next: [Ensemble representation](T7%20-%20Ensemble%20representation.ipynb)